In [1]:
from dotenv import load_dotenv

load_dotenv()

env: AWS_PROFILE=dev


In [2]:
!aws s3 ls

2024-01-04 05:42:12 bgp-ls-demo
2023-12-11 18:23:17 co-graph-drive-bucket-1
2023-06-05 05:14:22 deployed-ignatella-app-pets
2023-04-26 01:07:08 deployed-ignatella-tfstate
2024-03-18 12:22:52 ignatella-ray
2023-06-16 10:25:15 pt-ignatella-tfstate
2024-03-26 17:06:02 sagemaker-eu-north-1-422901541742
2024-03-26 14:51:07 sagemaker-studio-422901541742-45qvfhp8mmb
2024-03-26 14:39:15 sagemaker-studio-422901541742-dy8sadfa9t8
2024-03-26 17:05:58 sagemaker-studio-422901541742-ysuogi77x5e
2024-03-26 14:39:18 sagemaker-us-east-1-422901541742


In [11]:
import boto3

client = boto3.client('rekognition', region_name="eu-west-1")

with open('./iy-driving-licence.jpg', 'rb') as img:
    image_bytes = img.read()

response = client.detect_labels(Image={'Bytes': image_bytes})

response['Labels']

[{'Name': 'Text',
  'Confidence': 99.99976348876953,
  'Instances': [],
  'Parents': [],
  'Aliases': [],
  'Categories': [{'Name': 'Text and Documents'}]},
 {'Name': 'Document',
  'Confidence': 98.85714721679688,
  'Instances': [],
  'Parents': [{'Name': 'Text'}],
  'Aliases': [],
  'Categories': [{'Name': 'Text and Documents'}]},
 {'Name': 'Id Cards',
  'Confidence': 96.52450561523438,
  'Instances': [],
  'Parents': [{'Name': 'Document'}, {'Name': 'Text'}],
  'Aliases': [],
  'Categories': [{'Name': 'Text and Documents'}]},
 {'Name': 'Driving License',
  'Confidence': 95.9981689453125,
  'Instances': [{'BoundingBox': {'Width': 0.6353352665901184,
     'Height': 0.29270777106285095,
     'Left': 0.20309032499790192,
     'Top': 0.35422483086586},
    'Confidence': 93.89559936523438}],
  'Parents': [{'Name': 'Document'}, {'Name': 'Id Cards'}, {'Name': 'Text'}],
  'Aliases': [{'Name': 'License'}],
  'Categories': [{'Name': 'Text and Documents'}]}]

In [ ]:
# find label with name Driving License and get confidence

labels = [label for label in response['Labels'] if label['Name'] == 'Driving License']
label = labels[0] if labels else None

# find bounding box of the label

if label:
    box = label['Instances'][0]['BoundingBox']
    box

    # get image size
    from PIL import Image

    im = Image.open('./iy-driving-licence.jpg')
    width, height = im.size

    # calculate bounding box coordinates
    left = box['Left'] * width
    top = box['Top'] * height
    right = left + box['Width'] * width
    bottom = top + box['Height'] * height

    left, top, right, bottom

    # crop the image
    im.crop((left, top, right, bottom)).save('cropped.jpg')

    # show the cropped image
    from PIL import Image
    from IPython.display import display

    im = Image.open('cropped.jpg')
    display(im)